In [ ]:
import numpy as np
import pandas as pd
import os
import gc

import keras as k

import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from multiprocessing import Pool, cpu_count    

In [ ]:
#%matplotlib inline

In [ ]:
#np.empty((1, 256, 256, 3), dtype=np.float16).nbytes * 40000 / (1024*1024)

# 1. Data Preprocessing

In [ ]:
label_map = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'cultivation',
 'artisinal_mine',
 'haze',
 'primary',
 'slash_burn',
 'habitation',
 'clear',
 'road',
 'selective_logging',
 'partly_cloudy',
 'agriculture',
 'water',
 'cloudy']

inv_label_map = {'agriculture': 14,
 'artisinal_mine': 5,
 'bare_ground': 1,
 'blooming': 3,
 'blow_down': 0,
 'clear': 10,
 'cloudy': 16,
 'conventional_mine': 2,
 'cultivation': 4,
 'habitation': 9,
 'haze': 6,
 'partly_cloudy': 13,
 'primary': 7,
 'road': 11,
 'selective_logging': 12,
 'slash_burn': 8,
 'water': 15}

In [ ]:
img_height = 64
img_width  = 64

In [ ]:
df_train = pd.read_csv('../input/train_v2.csv')

flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

#key not same cannot save keras model
#label_map = {l: i for i, l in enumerate(labels)}
#inv_label_map = {i: l for l, i in label_map.items()}

Y = np.empty((df_train.shape[0]*5, 17), dtype=np.uint8)
i=0
for tags in tqdm(df_train['tags'].values, miniters=1000):
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    for j in range(i,i+5):
        Y[j,:] = targets
    i += 5
print(Y.shape)    

def get_images(names):
    i = 0
    X = np.empty((names.shape[0]*5, img_height, img_width, 3), dtype=np.float16)
    for f in tqdm(names.values, miniters=1000):
        img = cv2.imread('../input/train-jpg/{}.jpg'.format(f))
        if img_height != img.shape[0]:
            img = cv2.resize(img, (img_height, img_width))
        X[i,:,:,:] = np.array(img, np.float16)
        X[i+1,:,:,:] = np.array(cv2.flip(img, 0), np.float16)
        X[i+2,:,:,:] = np.array(cv2.flip(img, 1), np.float16)
        M = cv2.getRotationMatrix2D((img_height/2,img_width/2),90,1)
        X[i+3,:,:,:] = np.array(cv2.warpAffine(img, M, (img_height, img_width)))
        N = cv2.getRotationMatrix2D((img_height/2,img_width/2),-90,1)
        X[i+4,:,:,:] = np.array(cv2.warpAffine(img, N, (img_height, img_width)))
        i += 5
    return X / 255.

#multiply cpu_count if cannot fit memory
pool = Pool(cpu_count())
X = np.concatenate(pool.map(
    get_images, 
    np.array_split(df_train['image_name'], cpu_count())
))
pool.close()
pool.join()
print(X.shape)

# 2. Model Building

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
split = 35000
#x_train, x_valid, y_train, y_valid = X[:split], X[split:], Y[:split], Y[split:]
x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
def fbeta(y_true, y_pred):
    beta = 2
    threshold_shift = -0.3

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin), axis=1) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)), axis=1)
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)), axis=1)

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return K.mean((beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon()))

In [ ]:
train_generator = ImageDataGenerator(
                        rotation_range=180,
                        width_shift_range=0.2, 
                        height_shift_range=0.2,
                        shear_range=0.2,
                        zoom_range=2,
                    )

In [43]:
model = Sequential()
model.add(BatchNormalization(input_shape=(64, 64,3)))
model.add(Conv2D(32, kernel_size=(3, 3),padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3),padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3),padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, kernel_size=(3, 3),padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

In [44]:
opt  = Adam(lr=0.001)
model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
                  optimizer=opt,
                  metrics=['accuracy', fbeta])

In [ ]:
epochs_arr = [20, 5, 5]
learn_rates = [0.001, 0.0001, 0.00001]
kfold_weights_path = os.path.join('', 'weights.h5')

for learn_rate, epochs in zip(learn_rates, epochs_arr):
    opt  = Adam(lr=learn_rate)
    model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
                  optimizer=opt,
                  metrics=['accuracy', fbeta])
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=2, verbose=2),
        ModelCheckpoint(kfold_weights_path, monitor='val_loss', 
                        save_best_only=True, verbose=2)
    ]
    
    model.fit_generator(train_generator.flow(x_train, y_train, batch_size=128),
          steps_per_epoch=len(x_train) / 128,
          epochs=epochs,
          verbose=1,
          workers=3,
          validation_data=(x_valid, y_valid),
          callbacks=callbacks)

    #model.fit(x = x_train, y= y_train, validation_data=(x_valid, y_valid),
    #      batch_size=128,verbose=1, epochs=epochs,callbacks=callbacks,shuffle=True)

In [ ]:
#save!
model.save_weights('singlebest.h5')

In [45]:
#model.load_weights('singlebest.h5')

In [ ]:
from sklearn.metrics import fbeta_score, accuracy_score

In [ ]:
if os.path.isfile(kfold_weights_path):
    model.load_weights(kfold_weights_path)

In [ ]:
p_valid = model.predict(x_valid, batch_size=128, verbose=1)

In [ ]:
print(fbeta_score(y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))

In [ ]:
score = fbeta_score(y_valid, np.array(p_valid) > 0.2, beta=2, average=None)
print('F2 test scores per tag:')
[(inv_label_map[l], score[l]) for l in score.argsort()[::-1]]

In [ ]:
for i in range(17):
    print(inv_label_map[i], '\t:', accuracy_score(y_valid[:,i], p_valid[:,i]>0.2))

# 3. Make Prediction

In [ ]:
df_submission = pd.read_csv('../input/sample_submission_v2.csv')

def get_images(names):
    i = 0
    X = np.empty((names.shape[0], img_height, img_width, 3), dtype=np.float16)
    for f in tqdm(names.values, miniters=1000):
        img = cv2.imread('../input/test-jpg/{}.jpg'.format(f))
        if img_height != img.shape[0]:
            img = cv2.resize(img, (img_height, img_width))
        X[i,:,:,:] = np.array(img, np.float16)
        i += 1
    return X / 255.

pool = Pool(cpu_count())
X_submission = np.concatenate(pool.map(
    get_images, 
    np.array_split(df_submission['image_name'], cpu_count())
))
pool.close()
pool.join()
print(X_submission.shape)

In [ ]:
predict = model.predict(X_submission, batch_size = 128, verbose=1)

In [ ]:
result = pd.DataFrame(predict)
preds = []
sorted_tags = pd.Series(inv_label_map)

for i in tqdm(range(result.shape[0]), miniters=1000):
    preds.append(' '.join(list(
        sorted_tags[np.where(result.loc[i] == 1)[0]]
    )))

In [ ]:
df_submission['tags'] = preds
df_submission.to_csv('submission_best_augment.csv', index=False)